In [14]:
import numpy as np
import time
import math
import sympy as sp
import random
from itertools import permutations
from multiprocessing import Pool, freeze_support
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.cluster import KMeans

dataset = np.array([10.01769447, 10.06923099, 10.06665355,  9.98286628,  9.92245355, 10.01378089,
                   11.58835573 ,11.66084466 ,11.60612279, 11.65167558, 11.59958176 ,11.60105934,
 11.6644912 , 11.64641932 ,11.62979292, 11.66409849,11.64323261, 11.61256012,
 11.5894299 , 11.58006899, 11.65612268, 11.5998781 , 11.69453838 ,11.64668659,
 11.61357351, 11.60527658 ,11.66324746, 11.60354948 ,11.63437253, 11.64986473])

n_size = 30
T = 4
# 一共有T-1期，参与了T-2个factor（第0期不考虑）
num_e = T - 2
num_s = 3
num_a = 1

m_fix = np.ones(T)
m_fix = m_fix / 1
b = np.ones((T, num_a))
b = b / 1

c = np.ones((T, num_s))
c = c / 1

matrix_p = 0.1*np.ones((T, num_a, num_s))

mu_s = np.random.normal(1, 0, size=(T, num_s))
mu_a = np.random.normal(1, 0, size=(T, num_a))

In [29]:
def final(total):
    def policy(st,mu_a_t,mu_s_t,matrix_p_t):
        at = mu_a_t+ np.dot(matrix_p_t, st-mu_s_t)
        return at

    def reward(mt,bt,ct,at,st):
        rt = mt + np.dot(bt,at)+ np.dot(ct,st)
        return rt

    def f(st,at,mu_s_t1,mu_s_t,mu_a_t,beta_s_t, beta_a_t):
        s_t1 = mu_s_t1 + np.dot(beta_s_t, st-mu_s_t) + np.dot(beta_a_t, at-mu_a_t)
        return s_t1

    def P(perm, i):
        m_index = perm.index(i)
        if m_index > 0:
            # 切片，选择前面的所有元素
            ele_list = list(perm[0:m_index])
            return ele_list
        else:
            return list()

    def simulate_begin(e,sample_beta):
        beta_s[2][0] = np.ones(( num_s))*sample_beta*0.2
        beta_a[2][0] = np.ones((num_a))*sample_beta*0.2
        for t in range(1, T):
            a[t] = policy(s[t], mu_a[t], mu_s[t], matrix_p[t])
            if t + 1 < T:
                s[t + 1] = f(s[t], a[t], mu_s[t + 1], mu_s[t], mu_a[t], beta_s[t], beta_a[t]) + e[t-1]
            r[t] = reward(m_fix[t], b[t], c[t], a[t], s[t])
            R[t] = R[t - 1] + r[t]
        return R[T - 1]
   
    def g_hat(subset):
        if len(subset) == 0:
            return 0
        else:
            sample_size = 1000
            mu = 10 # 总体均值
            pos_value = np.zeros(sample_size)
            alpha_n, beta_n = update_pdf(subset) # 正常是每次从逆伽马分布中采样方差（我们暂时把均值当作真实方差）
            sigma2 = beta_n/(alpha_n-1)
            #print(np.sqrt(sigma2))
            # 现在我想用value function衡量后验与先验相比在reward方差上的改善
            for i in range(sample_size):
                e = np.random.normal(loc = 0 , scale = 0.1,size = 2)
                posterior = np.random.normal(loc = mu, scale = np.sqrt(sigma2))
                pos_value[i] = simulate_begin(e,posterior)
            res = 26.24 - np.var(pos_value, ddof =1)
            return res

    def generate_random_order(num):
        # 生成0到99的整数列表
        numbers = list(range(num))
        # 随机打乱列表
        random.shuffle(numbers)
        return numbers

    def update_pdf(subset): # 我知道beta来自于一个均值为10的正态，但我不知道方差是多少
        alpha_0 = 3 # 形状参数
        beta_0 = 8 # 尺度参数
        mu = 10 # 设置总体均值
        alpha_n = alpha_0 + len(subset)/2 # 更新形状参数
        beta_n = beta_0 + 0.5*np.sum([(i-mu)**2 for i in dataset[subset]])
        return alpha_n, beta_n

    def sample_sd(Sh, Sh2, N):
        over = Sh2 + Sh**2/N
        under = N-1
        return np.sqrt(over/under)

    def boundary_index(Sh_m, Sh_choose):
        bound = len(Sh_choose)
        i = np.argsort(Sh_m)[-bound]
        j = np.argsort(Sh_m)[-(bound+1)]
        return i,j

    # N是一个数组,这里暂定一共有6个数
    def budget_allocation(N,incre,std,gap):
        pp = sp.symbols('pp0:30')
        under = (std/gap)**2
        eq_sum = sp.Eq(sum(pp), np.sum(N)+incre)
        eqs = [sp.Eq(pp[i]/under[i], pp[0]/under[0]) for i in range(1, 30)]
        solution = sp.solve([eq_sum] + eqs, pp)
        N_star = np.zeros(30)
        # 这里要考虑四舍五入以及加和的问题
        for i in range(30):
            N_star[i] = round(float(solution[pp[i]]))
        # 多的就给最小的
        while np.sum(N_star) < np.sum(N) + incre:
            tmp = N_star/under
            i = np.argmin(tmp)
            N_star[i] = N_star[i] + 1
        return N_star

    # 优化问题的目标函数
    def objective(x):
        return np.sum([np.abs(x[i])for i in range(n_size)])

    # 约束条件
    def constraint(x):
        ep = 1/n_size
        tmp = np.zeros(M)
        for m in range(M):
            tmp[m] = np.sum([A[m][i]*(s_bar+x[i]) for i in range(n_size)]) - y_bar[m]
        return ep**2 - np.sum(tmp**2)


    '''
    一些主观设定
    '''
    N0 = 20
    b_incre = 100
    N = 1200
    correct = 0
    #total = 50
    true_set = [0,1,2,3,4,5]
    for _ in range(total):
        R = np.zeros(T)
        r = np.zeros(T)
        s = np.zeros((T, num_s))
        s[1] = 2*np.ones(num_s)
        a = np.zeros(T)
        Sh_hat = np.zeros(n_size)
        Sh2 = np.zeros(n_size)
        N_set = np.zeros(n_size)
        beta_s = np.ones((T, num_s, num_s))
        beta_a = np.ones((T, num_s, num_a))

        Sh_sample = dict()
        for i in range(n_size):
            Sh_sample[i] = list()
        
        
        for n in range(N0):
            perm = generate_random_order(n_size)
            for i in range(n_size):
                N_set[i] = N_set[i] + 1
                tmp = g_hat(P(perm,i)+[i]) - g_hat(P(perm,i))
                Sh_hat[i] = Sh_hat[i] + tmp
                Sh_sample[i].append(tmp)
                Sh2[i] = Sh2[i] + tmp**2
        
        
        '''
        二阶段部分
        '''
        while np.sum(N_set)< N:
            Sh_mean = Sh_hat/N_set
            #print(Sh_mean)
            Sh_std = sample_sd(Sh_hat, Sh2, N_set)
            sigma_hat = Sh_std/np.sqrt(N_set)
            Sh_sum = np.sum(Sh_mean)
            factor_choose = list()
            for i in range(n_size):
                if Sh_mean[i]>= 0.05*Sh_sum:
                    factor_choose.append(i)
            m, m1 = boundary_index(Sh_mean, factor_choose)
            boundary = (sigma_hat[m1]*Sh_mean[m] + sigma_hat[m]*Sh_mean[m1])/(sigma_hat[m] + sigma_hat[m1])
            gap = Sh_mean - boundary
            N_star = budget_allocation(N_set, b_incre, Sh_std, gap)
            incre_round = 0    
            while np.sum(N_set) < np.sum(N_star):
                if incre_round == b_incre:
                    break
                perm = generate_random_order(n_size)
                for i in range(n_size):
                    if incre_round < b_incre and N_set[i] < N_star[i]:
                        incre_round += 1
                        N_set[i] = N_set[i] + 1
                        tmp = g_hat(P(perm,i)+[i]) - g_hat(P(perm,i))
                        Sh_hat[i] = Sh_hat[i] + tmp
                        Sh_sample[i].append(tmp)
                        Sh2[i] = Sh2[i] + tmp**2
        '''
        '''
        M = 1300
        A = np.zeros((M,n_size))
        for i in range(M):
            for j in range(n_size):
                tmp  = random.random()
                if tmp < 0.5:
                    A[i][j] = -1/np.sqrt(M)
                else:
                    A[i][j] = 1/np.sqrt(M)
        d_max = int(np.max(N_set)) # 被赋予了最多样本的样本数
        Sh_mean = Sh_hat/N_set # 每个点的shapley均值
        Sh_fill = np.zeros((n_size, d_max))
        for i in range(n_size):
            for d in range(d_max):
                if d < N_set[i]:
                    Sh_fill[i][d] = Sh_sample[i][d]
                else:
                    Sh_fill[i][d] = Sh_mean[i]
        y = np.zeros((M,d_max))
        for m in range(M):
            for d in range(d_max):
                y[m][d] = np.sum([A[m][i]*Sh_fill[i][d] for i in range(n_size)])
        
        y_bar = np.zeros(M)
        for m in range(M):
            y_bar[m] = np.mean(y[m])
        
        
        
        
        Sh_mean = Sh_hat/N_set
        data =Sh_mean.reshape(-1,1) 
        kmeans = KMeans(n_clusters=2, random_state=42)
        kmeans.fit(data)

        # 获取聚类结果
        labels = kmeans.labels_

        # 根据标签将数据分为两类
        cluster_1 = data[labels == 0].flatten()
        cluster_2 = data[labels == 1].flatten()
        
        s_bar = np.mean(cluster_2)
        
        #s_bar = 0

        x0 = np.zeros(n_size)
        cons = {'type': 'ineq', 'fun': constraint}
        result = minimize(objective, x0, constraints=cons, method='SLSQP')
        Sh = np.array(result.x) + s_bar
        
        #Sh = Sh_hat/N_set
        Sh_sum = np.sum(Sh)
        factor_choose = list()
        for i in range(n_size):
            if Sh[i]>= 0.05*Sh_sum:
                factor_choose.append(i)

        if set(factor_choose)==set(true_set):
            correct = correct + 1
    return correct

In [30]:
if __name__ == '__main__':
    s_time = time.time()
    freeze_support()
    num_K = 64
    total = 5
    with Pool(processes= num_K) as pool:
        data = [total for i in range(num_K)]
        result = pool.map(final,data)
    res = 0
    for t in result:
        res += t
    print('准确率为：',res/(num_K*total))
    e_time = time.time()
    print('耗时：',e_time - s_time,'s')

准确率为： 0.721875
耗时： 2072.741450548172 s


In [ ]:
'''
900 1200 N0=10
1500 1800 N0 = 20
0.928125 M=1200
0.93125 M= 1500

0.878125
sequential
0.496875   0.753125 0.815625
0.5875  0.78125 0.846875

0.88125
sparse
1650 0.88125
'''